In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import RMSprop, Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from keras.layers.pooling import MaxPool2D
from tensorflow.keras.initializers import RandomNormal, Constant
from tensorflow.keras import utils
from tensorflow.keras.datasets import cifar10, cifar100

# importing of service libraries
import numpy as np
import matplotlib.pyplot as plt


2023-04-10 01:51:49.919958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import random
import tensorflow as tf
import os

def set_seed(seed=42):
    '''
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.
    '''
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [3]:
(input_X_train, output_y_train),(input_X_test, output_y_test)=cifar10.load_data()

print('input_X_train shape: ', input_X_train.shape)
print(input_X_train.shape[0], 'train samples')
print(input_X_test.shape[0], 'test samples')

input_X_train shape:  (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMAGE_SIZE = input_X_train.shape[1]
N_CLASSES =len(np.unique(output_y_train))

In [5]:
# output data one-hot encoding 
output_y_train=utils.to_categorical(output_y_train, N_CLASSES)
output_y_test=utils.to_categorical(output_y_test, N_CLASSES)

In [6]:
# To normalize the value in between 0 and 1 (there are 255 kinds)
input_X_train=input_X_train.astype('float32')
input_X_test=input_X_test.astype('float32')

input_X_train /=255
input_X_test /=255

In [8]:
#Define the function for plotting the history of the training of the model
def plot_history(history):
    val_loss = history.history['val_loss' ]
    loss =     history.history['loss' ]
    acc =      history.history['accuracy' ]
    val_acc =  history.history['val_accuracy' ]

    epochs    = range(1,len(acc)+1,1)

    plt.plot  ( epochs,     acc, 'r--', label='Training acc'  )
    plt.plot  ( epochs, val_acc,  'b', label='Validation acc')
    plt.title ('Training and validation accuracy')
    plt.ylabel('acc')
    plt.xlabel('epochs')
    plt.legend()

    plt.figure()

    plt.plot  ( epochs,     loss, 'r--', label='Training loss' )
    plt.plot  ( epochs, val_loss ,  'b', label='Validation loss' )
    plt.title ('Training and validation loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend()

    plt.figure()

In [13]:
# Define the hyperparameter grid
param_grid = {
    'epochs': [20, 30, 40, 50],
    'learning_rate': [1, 0.1, 0.01, 0.001, 0.0001],
    'batch_size': [128, 256, 512],
    'optimizer': [RMSprop(), Adam(), SGD()],
    'validation_split': [0.2, 0.3, 0.4, 0.5],
    'model': ['Simple CNN', 'Complex CNN', 'CNN with BN'],
    'kernel_size': [(3,3), (5,5)],
    'data_augmentation': [True, False]
}

In [14]:
# Define the function to create the Keras model
def create_model(optimizer, kernel_size='kernel_size', model='model', batch_size='batch_size', learning_rate="learning_rate", epochs='epochs', validation_split='validation_split'):
    if model == 'Simple CNN':
        model = Sequential()
        model.add(Conv2D(32, kernel_size, padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, IMG_CHANNELS)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(N_CLASSES))
        model.add(Activation('softmax'))
    elif model == 'Complex CNN':
        # Define the architecture of the Complex CNN model
        # Complex DNN model definition
        model = Sequential()

        # hidden 1 : conv + conv + pool + dropout 
        model.add(Conv2D(32, kernel_size, padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, IMG_CHANNELS)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, kernel_size, padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # hidden 2 : conv + conv + pool + dropout 
        model.add(Conv2D(64, kernel_size, padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(64, 3, 3))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # hidden 3 : flatten + droupout 
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))

        # output 
        model.add(Dense(N_CLASSES))
        model.add(Activation('softmax'))
        pass
    elif model == 'CNN with BN':
        # Define the architecture of the CNN with BN model
        model = Sequential()
        model.add(Conv2D(256,kernel_size,padding='same',input_shape=(IMAGE_SIZE, IMAGE_SIZE, IMG_CHANNELS)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Conv2D(256,kernel_size,padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.2))
        
        model.add(Conv2D(512,kernel_size,padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Conv2D(512,kernel_size,padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.2))

        model.add(Conv2D(512,kernel_size,padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Conv2D(512,kernel_size,padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.2))

        model.add(Conv2D(512,kernel_size,padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Conv2D(512,kernel_size,padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.2))

        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization(momentum=0.95, 
                epsilon=0.005,
                beta_initializer=RandomNormal(mean=0.0, stddev=0.05), 
                gamma_initializer=Constant(value=0.9)))
        model.add(Dense(100,activation='softmax'))
        pass
    model.compile(loss='categorical_crossentropy', optimizer=optimizer(learning_rate='learning_rate'), metrics=['accuracy'])
    history=model.fit(input_X_train, output_y_train, batch_size='batch_size', 
                  epochs='epochs', validation_split='validation_split', verbose=1)


In [12]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_X_train, output_y_train, test_size=0.2)

# Define the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
    x=X_train,
    y=y_train,
    batch_size='batch_size',
    shuffle=True
)

validation_generator = test_datagen.flow(
    x=X_val,
    y=y_val,
    batch_size='batch_size',
    shuffle=True
)

In [15]:
# Define the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
    x=input_X_train,
    y=output_y_train,
    batch_size='batch_size',
    shuffle=True
)

In [16]:
# Wrap the Keras model with KerasClassifier from sklearn
keras_clf = KerasClassifier(build_fn=create_model)

# Create a GridSearchCV object to tune the hyperparameters
grid_search = GridSearchCV(estimator=keras_clf, param_grid=param_grid, cv=3)

# Train the model with GridSearchCV
# grid_search.fit(train_generator, epochs=10, validation_data=validation_generator)
grid_search.fit(X=input_X_train, y=output_y_train, epochs=10, validation_split=0.2)


/var/folders/0x/n51_7s_n09dfndljqk4rv2zr0000gn/T/ipykernel_73180/2803159475.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_clf = KerasClassifier(build_fn=create_model)
